In [ ]:
import pandas as pd
import sys
import numpy as np

# 파일 경로 설정
luad_genomics_file_path = '/shared/js.yun/data/CLAM_data/genomics_data/luad_data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt'
lusc_genomics_file_path = '/shared/js.yun/data/CLAM_data/genomics_data/lusc_data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt'

gene_family_file_path = '/shared/js.yun/data/CLAM_data/genomics_data/gene_family.csv'

# 파일을 DataFrame으로 읽어오기
df_luad_zscore = pd.read_csv(luad_genomics_file_path, delimiter='\t')   # 각 row는 Hugo_Symbol이고 column은 환자
df_lusc_zscore = pd.read_csv(lusc_genomics_file_path, delimiter='\t')   # 각 row는 Hugo_Symbol이고 column은 환자


df_gene_family = pd.read_csv(gene_family_file_path)     # column이 gene family고 그 구성에 해당하는 Hugo_Symbol이 적혀있음
# DataFrame 출력
# print(df_genomics)
# print(df_gene_family)

# DataFrame을 Python 리스트로 변환
family_genes = df_gene_family.stack().dropna().unique().tolist()

# 중복해서 3860개
# 중복 1014

print(family_genes)
print(len(family_genes))                    # 중복 안되는 gene 3051개
print(len(df_luad_zscore.columns[2:]))    # LUAD 510명
print(len(df_lusc_zscore.columns[2:]))    # LUSC 484명

In [ ]:
# family_genes 리스트에 속해있는 row의 index 추출
selected_gene_indices_luad = [i for i, row_name in enumerate(df_luad_zscore.iloc[:,0]) if row_name in family_genes]
selected_gene_indices_lusc = [i for i, row_name in enumerate(df_lusc_zscore.iloc[:,0]) if row_name in family_genes]
# family_genes 리스트에 속해있는 row의 gene name 추출 -> 어떤 genen이 중복 되는지 확인용
selected_gene_name_luad = [row_name for _, row_name in enumerate(df_luad_zscore.iloc[:,0]) if row_name in family_genes]
selected_gene_name_lusc = [row_name for _, row_name in enumerate(df_lusc_zscore.iloc[:,0]) if row_name in family_genes]
# print(selected_gene_indices_luad)
# print(selected_gene_indices_lusc)
# print(selected_gene_indices_luad == selected_gene_indices_lusc)     # True


# # 전체 df_luad_zscore에서 중복된 값 확인 -> gene family에 없는게 대부분이라 필요 없음
# duplicates = df_luad_zscore[df_luad_zscore.duplicated(subset=[df_luad_zscore.columns[0]], keep=False)]
# # 중복된 값 출력
# print(duplicates[df_luad_zscore.columns[0]].unique())

# gene family에 속하는 gene 중 중복된 값 확인
from collections import Counter
duplicates_luad = [item for item, count in Counter(selected_gene_name_luad).items() if count > 1]
duplicates_lusc = [item for item, count in Counter(selected_gene_name_lusc).items() if count > 1]
if len(duplicates_luad) > 0:
    print("LUAD 중복된 값:", duplicates_luad)   # FGF13
else:
    print("LUAD 중복된 값 없음")
if len(duplicates_lusc) > 0:
    print("LUSC 중복된 값:", duplicates_lusc)   # FGF13
else:
    print("LUSC 중복된 값 없음")

# gene family 중에서 data에 없는 gene list: LUAD나 LUSC나 동일
not_exist_gene_luad = [gene for gene in family_genes if gene not in df_luad_zscore.iloc[:,0].tolist()]
not_exist_gene_lusc = [gene for gene in family_genes if gene not in df_lusc_zscore.iloc[:,0].tolist()]
# print(not_exist_gene_luad)
# print(not_exist_gene_lusc)
# print(not_exist_gene_luad == not_exist_gene_lusc)       # True
print(f'Number of genes in gene family (LUAD): {len(selected_gene_indices_luad)}')
print(f'Number of genes that do not exist (LUAD): {len(not_exist_gene_luad)}')          
print(f'Number of genes in gene family (LUSC): {len(selected_gene_indices_lusc)}')
print(f'Number of genes that do not exist (LUSC): {len(not_exist_gene_lusc)}')          

# gene family에 속하는 gene만 dataframe으로 만듦
df_result_luad = df_luad_zscore.iloc[selected_gene_indices_luad,2:].reset_index(drop=True)
df_result_lusc = df_lusc_zscore.iloc[selected_gene_indices_lusc,2:].reset_index(drop=True)

# 삭제 전 인덱스
indices_before_luad = df_result_luad.index.tolist()  # 삭제 전의 인덱스
indices_before_lusc = df_result_lusc.index.tolist()  # 삭제 전의 인덱스
# NaN 값을 가진 행 제거
df_result_luad_temp = df_result_luad.dropna()
df_result_lusc_temp = df_result_lusc.dropna()
# 삭제 후 인덱스
indices_after_luad = df_result_luad_temp.index.tolist()  # 삭제 후의 인덱스
indices_after_lusc = df_result_lusc_temp.index.tolist()  # 삭제 후의 인덱스
# 삭제된 행의 인덱스 찾기
deleted_indices_luad = list(set(indices_before_luad) - set(indices_after_luad))
deleted_indices_lusc = list(set(indices_before_lusc) - set(indices_after_lusc))
deleted_indices_luad.sort()
deleted_indices_lusc.sort()
print("\nLUAD 삭제된 행의 인덱스:", deleted_indices_luad)
print("LUSC 삭제된 행의 인덱스:", deleted_indices_lusc)
deleted_indices_union = list(set(deleted_indices_luad+deleted_indices_lusc))
print("Union: ", deleted_indices_union)
# from IPython.display import display
# display(df_result_luad)
# display(df_result_lusc)

print(len(indices_before_luad))
print(len(indices_after_luad))
print(len(deleted_indices_union))

NaN 값으로만 채워진 gene이 있는데, LUAD랑 LUSC가 그 종류가 다름. LUAD는 NA로만 채워진 row가 더 많음

### *LUAD나 LUSC 중 하나라도 NA이면 그 row는 사용하지 않는 걸로*

In [ ]:
df_luad_2847 = df_result_luad.drop(deleted_indices_union).reset_index(drop=True)
df_lusc_2847 = df_result_lusc.drop(deleted_indices_union).reset_index(drop=True)

display(df_luad_2847)
display(df_lusc_2847)

# 이미지가 없는 genomics 제거

In [86]:
patient_list_genomics_luad = df_luad_2847.columns.tolist()
patient_list_genomics_lusc = df_lusc_2847.columns.tolist()

# LUSC랑 LUAD 환자 중에 겹치는게 있는지 확인 - 중복 되는게 있으면 
set1 = set(patient_list_genomics_luad)
set2 = set(patient_list_genomics_lusc)
intersection = set1.intersection(set2)
if intersection:
    print("LUSC, LUAD genomics 간에 겹치는 요소가 있습니다.")
    print("LUSC, LUAD genomics 교집합:", list(intersection))
else:
    print("LUSC, LUAD genomics 간에 겹치는 요소가 없습니다.")


# feature 데이터셋 폴더 경로
folder_path = '/shared/j.jang/pathai/data/TCGA-lung-x256-features-dino-from-pretrained-vitb-img224/h5_files'
# 파일 이름 가져오기
import os
file_names = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.startswith('TCGA'):
            file_names.append(file)
# WSI 환자 리스트
patient_list_image = ['-'.join(file.split('-')[:3]+[file.split('-')[3][:2]]) for file in file_names]
# patient_list_image = [file.split('.')[0] for file in file_names]
# print('images patient', patient_list_image)


# 이미지에서 2번 이상 나온 환자 있는지 확인
patientwise_list_image = ['-'.join(file.split('-')[:3]) for file in file_names]
image_patient_count = Counter(patientwise_list_image)
duplicated_elements = [element for element, count in image_patient_count.items() if count >= 2]
# print(image_patient_count)
count = 0
for key, value in image_patient_count.items():
    if value >= 2:
        # print(f"{key}: {value}\t")
        count += 1
    # if 'TCGA-50-5066' in key:
    #     print(key)  
print(f'{count} patients have duplicated WSIs')


# genomics LUAD, LUSC 중 WSI 있는 애들만 선별
luad_intersection = []
lusc_intersection = []
for patient in patient_list_genomics_luad:
    if patient in patient_list_image:
        luad_intersection.append(patient)
for patient in patient_list_genomics_lusc:
    if patient in patient_list_image:
        lusc_intersection.append(patient)
print(len(luad_intersection))
print(len(lusc_intersection))


# print('WSI, genomics 모두 있는 사람:', len(patient_intersection))



LUSC, LUAD genomics 간에 겹치는 요소가 없습니다.
31 patients have duplicated WSIs
462
445


In [89]:
# 최대 열 수 설정 (None으로 설정하면 모든 열이 표시됨)
pd.set_option('display.max_columns', None)

# # 최대 행 수 설정 (None으로 설정하면 모든 행이 표시됨)
# pd.set_option('display.max_rows', None)

# genomics랑 WSI에 모두 있는 환자만 남기기
df_luad_2847 = df_luad_2847.loc[:, luad_intersection]
df_lusc_2847 = df_lusc_2847.loc[:, lusc_intersection]

df_luad_lusc_2847 = pd.concat([df_luad_2847, df_lusc_2847], axis=1)
display(df_luad_lusc_2847.head())
# print(len(df_luad_lusc_2847.columns.tolist()))


,TCGA-05-4244-01,TCGA-05-4249-01,TCGA-05-4250-01,TCGA-05-4382-01,TCGA-05-4384-01,TCGA-05-4390-01,TCGA-05-4395-01,TCGA-05-4396-01,TCGA-05-4397-01,TCGA-05-4398-01,TCGA-05-4402-01,TCGA-05-4403-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4415-01,TCGA-05-4417-01,TCGA-05-4418-01,TCGA-05-4420-01,TCGA-05-4422-01,TCGA-05-4424-01,TCGA-05-4425-01,TCGA-05-4426-01,TCGA-05-4427-01,TCGA-05-4430-01,TCGA-05-4432-01,TCGA-05-4433-01,TCGA-05-4434-01,TCGA-05-5423-01,TCGA-05-5425-01,TCGA-05-5428-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-35-3615-01,TCGA-35-4122-01,TCGA-35-4123-01,TCGA-35-5375-01,TCGA-38-4625-01,TCGA-38-4626-01,TCGA-38-4627-01,TCGA-38-4628-01,TCGA-38-4631-01,TCGA-38-4632-01,TCGA-38-6178-01,TCGA-38-7271-01,TCGA-38-A44F-01,TCGA-44-2655-01,TCGA-44-2656-01,TCGA-44-2657-01,TCGA-44-2659-01,TCGA-44-2662-01,TCGA-44-2665-01,TCGA-44-2666-01,TCGA-44-2668-01,TCGA-44-3396-01,TCGA-44-3398-01,TCGA-44-3918-01,TCGA-44-3919-01,TCGA-44-4112-01,TCGA-44-5643-01,TCGA-44-5644-01,TCGA-44-5645-01,TCGA-44-6145-01,TCGA-44-6146-01,TCGA-44-6148-01,TCGA-44-6774-01,TCGA-44-6775-01,TCGA-44-6776-01,TCGA-44-6777-01,TCGA-44-6778-01,TCGA-44-6779-01,TCGA-44-7659-01,TCGA-44-7660-01,TCGA-44-7661-01,TCGA-44-7662-01,TCGA-44-7667-01,TCGA-44-7669-01,TCGA-44-7670-01,TCGA-44-7671-01,TCGA-44-7672-01,TCGA-44-8119-01,TCGA-44-A479-01,TCGA-44-A47A-01,TCGA-44-A47B-01,TCGA-44-A47G-01,TCGA-44-A4SS-01,TCGA-44-A4SU-01,TCGA-49-4487-01,TCGA-49-4488-01,TCGA-49-4490-01,TCGA-49-4494-01,TCGA-49-4501-01,TCGA-49-4505-01,TCGA-49-4506-01,TCGA-49-4507-01,TCGA-49-4510-01,TCGA-49-4512-01,TCGA-49-4514-01,TCGA-49-6742-01,TCGA-49-6743-01,TCGA-49-6744-01,TCGA-49-6745-01,TCGA-49-6761-01,TCGA-49-6767-01,TCGA-49-AAQV-01,TCGA-49-AAR0-01,TCGA-49-AAR2-01,TCGA-49-AAR3-01,TCGA-49-AAR4-01,TCGA-49-AAR9-01,TCGA-49-AARE-01,TCGA-49-AARN-01,TCGA-49-AARO-01,TCGA-49-AARQ-01,TCGA-49-AARR-01,TCGA-4B-A93V-01,TCGA-50-5044-01,TCGA-50-5045-01,TCGA-50-5055-01,TCGA-50-5066-01,TCGA-50-5068-01,TCGA-50-5931-01,TCGA-50-5939-01,TCGA-50-5942-01,TCGA-50-5946-01,TCGA-50-6590-01,TCGA-50-6591-01,TCGA-50-6593-01,TCGA-50-6594-01,TCGA-50-6597-01,TCGA-50-6673-01,TCGA-50-7109-01,TCGA-50-8457-01,TCGA-50-8459-01,TCGA-53-7624-01,TCGA-53-7626-01,TCGA-53-7813-01,TCGA-53-A4EZ-01,TCGA-55-1592-01,TCGA-55-1594-01,TCGA-55-1595-01,TCGA-55-1596-01,TCGA-55-5899-01,TCGA-55-6543-01,TCGA-55-6642-01,TCGA-55-6712-01,TCGA-55-6968-01,TCGA-55-6969-01,TCGA-55-6970-01,TCGA-55-6971-01,TCGA-55-6972-01,TCGA-55-6975-01,TCGA-55-6978-01,TCGA-55-6979-01,TCGA-55-6980-01,TCGA-55-6981-01,TCGA-55-6982-01,TCGA-55-6983-01,TCGA-55-6984-01,TCGA-55-6985-01,TCGA-55-6986-01,TCGA-55-6987-01,TCGA-55-7227-01,TCGA-55-7281-01,TCGA-55-7283-01,TCGA-55-7284-01,TCGA-55-7570-01,TCGA-55-7573-01,TCGA-55-7574-01,TCGA-55-7576-01,TCGA-55-7724-01,TCGA-55-7725-01,TCGA-55-7726-01,TCGA-55-7727-01,TCGA-55-7728-01,TCGA-55-7815-01,TCGA-55-7816-01,TCGA-55-7903-01,TCGA-55-7907-01,TCGA-55-7910-01,TCGA-55-7911-01,TCGA-55-7913-01,TCGA-55-7914-01,TCGA-55-7994-01,TCGA-55-7995-01,TCGA-55-8085-01,TCGA-55-8087-01,TCGA-55-8089-01,TCGA-55-8090-01,TCGA-55-8091-01,TCGA-55-8092-01,TCGA-55-8094-01,TCGA-55-8096-01,TCGA-55-8097-01,TCGA-55-8203-01,TCGA-55-8204-01,TCGA-55-8205-01,TCGA-55-8206-01,TCGA-55-8208-01,TCGA-55-8299-01,TCGA-55-8301-01,TCGA-55-8302-01,TCGA-55-8505-01,TCGA-55-8506-01,TCGA-55-8507-01,TCGA-55-8508-01,TCGA-55-8510-01,TCGA-55-8511-01,TCGA-55-8512-01,TCGA-55-8513-01,TCGA-55-8514-01,TCGA-55-8614-01,TCGA-55-8615-01,TCGA-55-8616-01,TCGA-55-8619-01,TCGA-55-8620-01,TCGA-55-8621-01,TCGA-55-A48X-01,TCGA-55-A48Y-01,TCGA-55-A48Z-01,TCGA-55-A490-01,TCGA-55-A491-01,TCGA-55-A492-01,TCGA-55-A493-01,TCGA-55-A494-01,TCGA-55-A4DF-01,TCGA-55-A4DG-01,TCGA-55-A57B-01,TCGA-62-A46O-01,TCGA-62-A46P-01,TCGA-62-A46R-01,TCGA-62-A46S-01,TCGA-62-A46U-01,TCGA-62-A46V-01,TCGA-62-A470-01,TCGA-62-A471-01,TCGA-62-A472-01,TCGA-64-1676-01,TCGA-64-1677-01,TCGA-64-1679-01,TCGA-64-1680-01,TCGA-64-1681-01,TCGA-64-5774-01,TCGA-64-5775-01,TCGA-64-5778-01,TCGA-64-5779-01,TCGA-64-5781-01,TCGA-64-5815-01,TCGA-67-3770-01,TCGA-67-3771-01,TCGA-67-3772-01

907


In [90]:
df_luad_2847.to_csv('/shared/js.yun/data/CLAM_data/genomics_data/TCGA-lung-LUAD-selected_2847_zscore.csv', index=False)
df_lusc_2847.to_csv('/shared/js.yun/data/CLAM_data/genomics_data/TCGA-lung-LUSC-selected_2847_zscore.csv', index=False)
df_luad_lusc_2847.to_csv('/shared/js.yun/data/CLAM_data/genomics_data/TCGA-lung-LUAD+LUSC-selected_2847_zscore.csv', index=False)